In [2]:
#%matplotlib inline
#import matplotlib.pyplot as plt
#import graphviz
import numpy as np
import pandas as pd
from sklearn import preprocessing
import xgboost as xgb
import kaggle_xgb
import calc_ndcg
#from sklearn.grid_search import ParameterGrid
import pickle
import dataEngr
reload(dataEngr)

<module 'dataEngr' from 'dataEngr.pyc'>

Read in just the testing and training data.

In [ ]:
xgbInput = dataEngr.clfInput()
xgbInput.get_sessionsFtr()
xgbInput.users_ftrEng()
xgbInput.one_hot()
#xgbInput.binarize_targets()
xgbInput.split_data(update_trainDf=True)

In [ ]:
%run -i custom_paramsearch.py

number of columns before one hot encoding 370
number of columns after one hot encoding 469
15

[0]	eval-error:0.155979
[1]	eval-error:0.153003
[2]	eval-error:0.155197
[3]	eval-error:0.154204
[4]	eval-error:0.153658
[5]	eval-error:0.151832
[6]	eval-error:0.151362
[7]	eval-error:0.150855
[8]	eval-error:0.150198
[9]	eval-error:0.150452
[10]	eval-error:0.151138
[11]	eval-error:0.151200
[12]	eval-error:0.150859
[13]	eval-error:0.150898
[14]	eval-error:0.150317
[15]	eval-error:0.150496
[16]	eval-error:0.150266
[17]	eval-error:0.150039
[18]	eval-error:0.150156
[19]	eval-error:0.149701
[20]	eval-error:0.149570
[21]	eval-error:0.148937
[22]	eval-error:0.148301
[23]	eval-error:0.149242
[24]	eval-error:0.148908
[25]	eval-error:0.148358
[26]	eval-error:0.147466
[27]	eval-error:0.147397
[28]	eval-error:0.147502
[29]	eval-error:0.147495
[30]	eval-error:0.147566
[31]	eval-error:0.147330
[32]	eval-error:0.146907
[33]	eval-error:0.146344
[34]	eval-error:0.146182
[35]	eval-error:0.145787
[36]	eval-error:0.146490
[37]	eval-error:0.145880
[38]	eval-error:0.145680
[39]	eval-error:0.145040
[40]	eval-

In [ ]:
%run -i custom_paramsearch.py

In [ ]:
with open('../xgbmodels/actions2_e13_96n.p') as f:
    bst = pickle.load(f)

In [ ]:
features = sorted(bst.get_fscore().items(), key = lambda x: x[1], reverse = True)
[(feat, xgbInput.allDf.columns[int(feat[1:])], score) for feat, score in features]